In [ ]:
import gradio as gr
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, date
import sqlite3

# Common food dictionary

common_foods = {
    "Banana": 105,
    "Apple": 95,
    "Egg": 78,
    "Chicken Breast (100g)": 165,
    "Rice (1 cup)": 205,
    "Avocado": 240,
    "Cheeseburger": 303
}

def get_db_connection():
    conn = sqlite3.connect('health_data.db')
    conn.row_factory = sqlite3.Row
    return conn

def initialize_database():
    """Creates necessary tables if they do not exist."""
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS bmi_records (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            date TEXT,
            height_value REAL,
            height_unit TEXT,
            weight_value REAL,
            weight_unit TEXT,
            bmi REAL,
            category TEXT
        )
    ''')
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS metabolic_rates (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            date TEXT,
            age INTEGER,
            gender TEXT,
            height_value REAL,
            height_unit TEXT,
            weight_value REAL,
            weight_unit TEXT,
            activity_level TEXT,
            bmr REAL,
            tdee REAL
        )
    ''')
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS food_log (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            date TEXT,
            food_item TEXT,
            calories REAL
        )
    ''')
    conn.commit()
    conn.close()


initialize_database()

# BMI Calculator Functions

def convert_to_metric(height_val, height_unit, weight_val, weight_unit):
    if height_unit == "inches":
        height_m = height_val * 0.0254
    else:
        height_m = height_val / 100
    if weight_unit == "lbs":
        weight_kg = weight_val * 0.453592
    else:
        weight_kg = weight_val
    return height_m, weight_kg

def calculate_bmi(height_val, height_unit, weight_val, weight_unit, date_str):
    conn = get_db_connection()
    cursor = conn.cursor()

    height_m, weight_kg = convert_to_metric(height_val, height_unit, weight_val, weight_unit)

    if height_m <= 0 or weight_kg <= 0:
        conn.close()
        return "Error: Height and Weight must be positive numbers.", "", "", None

    try:
        date_obj = datetime.strptime(date_str, "%Y-%m-%d").date()
    except ValueError:
        conn.close()
        return "Error: Invalid date format. Use YYYY-MM-DD.", "", "", None

    bmi = weight_kg / (height_m ** 2)

    if bmi < 18.5:
        category = "Underweight"
        indicator = "🟦 Underweight"
    elif 18.5 <= bmi < 25:
        category = "Normal"
        indicator = "🟩 Normal"
    elif 25 <= bmi < 30:
        category = "Overweight"
        indicator = "🟨 Overweight"
    else:
        category = "Obesity"
        indicator = "🟥 Obese"

    cursor.execute('''
        INSERT INTO bmi_records (date, height_value, height_unit, weight_value, weight_unit, bmi, category)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', (date_obj.strftime("%Y-%m-%d"), height_val, height_unit, weight_val, weight_unit, bmi, category))
    conn.commit()

    cursor.execute('SELECT date, bmi FROM bmi_records ORDER BY date')
    historical_data = cursor.fetchall()
    conn.close()

    if historical_data:
        dates = [datetime.strptime(row['date'], "%Y-%m-%d") for row in historical_data]
        values = [row['bmi'] for row in historical_data]
        plt.figure(figsize=(6, 3))
        plt.plot(dates, values, marker='o', linestyle='-', color='blue')
        plt.title("BMI Over Time")
        plt.xlabel("Date")
        plt.ylabel("BMI")
        plt.xticks(rotation=45)
        plt.grid(True)
        plt.tight_layout()
        bmi_plot = plt.gcf()
    else:
        bmi_plot = None

    return round(bmi, 2), category, indicator, bmi_plot

# Metabolic Rate Functions

def calculate_bmr(age, gender, height_val, height_unit, weight_val, weight_unit, activity_level):
    conn = get_db_connection()
    cursor = conn.cursor()

    height_cm = height_val * 2.54 if height_unit == "inches" else height_val
    weight_kg = weight_val * 0.453592 if weight_unit == "lbs" else weight_val

    if age <= 0 or height_cm <= 0 or weight_kg <= 0:
        conn.close()
        return "Error: Age, Height, and Weight must be positive numbers.", "", "", "", ""

    if gender == "Male":
        bmr = 88.362 + (13.397 * weight_kg) + (4.799 * height_cm) - (5.677 * age)
    else:
        bmr = 447.593 + (9.247 * weight_kg) + (3.098 * height_cm) - (4.330 * age)

    activity_multipliers = {
        "Sedentary (little to no exercise)": 1.2,
        "Lightly active (1-3 days/week)": 1.375,
        "Moderately active (3-5 days/week)": 1.55,
        "Very active (6-7 days/week)": 1.725,
        "Extra active (daily, physical job)": 1.9
    }
    tdee = bmr * activity_multipliers.get(activity_level, 1.2)

    current_date = date.today().strftime("%Y-%m-%d")
    cursor.execute('''
        INSERT INTO metabolic_rates (date, age, gender, height_value, height_unit, weight_value, weight_unit, activity_level, bmr, tdee)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (current_date, age, gender, height_val, height_unit, weight_val, weight_unit, activity_level, bmr, tdee))
    conn.commit()
    conn.close()

    return round(bmr, 2), round(tdee, 2), round(tdee - 500, 2), round(tdee, 2), round(tdee + 500, 2)


# Food Tracker

def log_food(food_item, calories_to_add, date_str, tdee_from_tab2):
    conn = get_db_connection()
    cursor = conn.cursor()

    try:
        date_obj = datetime.strptime(date_str, "%Y-%m-%d").date()
    except ValueError:
        conn.close()
        return [], 0, "Error: Invalid date format. Use YYYY-MM-DD.", None

    if calories_to_add is None or calories_to_add == 0:
        calories_to_add = common_foods.get(food_item, 0)

    cursor.execute('''
        INSERT INTO food_log (date, food_item, calories)
        VALUES (?, ?, ?)
    ''', (date_obj.strftime("%Y-%m-%d"), food_item or "Manual Entry", calories_to_add))
    conn.commit()

    cursor.execute('SELECT date, food_item, calories FROM food_log ORDER BY date DESC')
    historical_log = cursor.fetchall()
    conn.close()

    daily_total = sum(row['calories'] for row in historical_log if datetime.strptime(row['date'], "%Y-%m-%d").date() == date_obj)

    comparison = "N/A (TDEE not provided)"
    if tdee_from_tab2 and tdee_from_tab2 > 0:
        comparison = "Under TDEE ✅" if daily_total < tdee_from_tab2 else "Over TDEE ⚠️"

    today = date.today()
    past_week = [today - timedelta(days=i) for i in range(6, -1, -1)]
    totals = [sum(row['calories'] for row in historical_log if datetime.strptime(row['date'], "%Y-%m-%d").date() == d) for d in past_week]

    plt.figure(figsize=(6, 3))
    plt.bar([d.strftime("%m-%d") for d in past_week], totals, color='skyblue')
    plt.title("Calories in Past 7 Days")
    plt.xlabel("Date")
    plt.ylabel("Calories")
    plt.tight_layout()
    weekly_plot = plt.gcf()

    dataframe_data = [[row['date'], row['food_item'], row['calories']] for row in historical_log]
    return dataframe_data, daily_total, comparison, weekly_plot


# Data Trends

def get_bmi_data():
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('SELECT date, height_value, height_unit, weight_value, weight_unit, bmi, category FROM bmi_records ORDER BY date')
    data = cursor.fetchall()
    conn.close()
    return [[row['date'], row['height_value'], row['height_unit'], row['weight_value'], row['weight_unit'], row['bmi'], row['category']] for row in data]

def get_metabolic_data():
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('SELECT date, age, gender, height_value, height_unit, weight_value, weight_unit, activity_level, bmr, tdee FROM metabolic_rates ORDER BY date')
    data = cursor.fetchall()
    conn.close()
    return [[row['date'], row['age'], row['gender'], row['height_value'], row['height_unit'], row['weight_value'], row['weight_unit'], row['activity_level'], row['bmr'], row['tdee']] for row in data]

def get_food_log_data():
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('SELECT date, food_item, calories FROM food_log ORDER BY date DESC')
    data = cursor.fetchall()
    conn.close()
    return [[row['date'], row['food_item'], row['calories']] for row in data]

# Gradio Interface

with gr.Blocks(title="Health Dashboard") as demo:
    gr.Markdown("# 🧠 My Health Monitoring Dashboard")
    gr.Markdown("Track BMI, BMR, TDEE, and calories with a persistent local database.")

    with gr.Tabs():
        # TAB 1: BMI
        with gr.Tab("BMI Calculator"):
            gr.Markdown("### Body Mass Index (BMI) Calculator")

            with gr.Row():
                height_input = gr.Number(label="Height Value")
                height_unit_select = gr.Radio(choices=["cm", "inches"], label="Height Unit", value="cm")

            with gr.Row():
                weight_input = gr.Number(label="Weight Value")
                weight_unit_select = gr.Radio(choices=["kg", "lbs"], label="Weight Unit", value="kg")

            date_input_bmi = gr.Textbox(label="Date (YYYY-MM-DD)", value=date.today().strftime("%Y-%m-%d"))
            submit_bmi_button = gr.Button("Calculate BMI")
            bmi_value_output = gr.Number(label="BMI Value")
            bmi_category_output = gr.Text(label="Category")
            bmi_indicator_output = gr.Text(label="Indicator")
            bmi_plot_output = gr.Plot(label="BMI Chart")

            submit_bmi_button.click(
                fn=calculate_bmi,
                inputs=[height_input, height_unit_select, weight_input, weight_unit_select, date_input_bmi],
                outputs=[bmi_value_output, bmi_category_output, bmi_indicator_output, bmi_plot_output]
            )

        # TAB 2: Metabolic Rate
        with gr.Tab("Metabolic Rate"):
            gr.Markdown("### Basal & Total Daily Energy Expenditure (BMR/TDEE) Calculator")

            with gr.Row():
                age_input = gr.Number(label="Age")
                gender_input = gr.Radio(choices=["Male", "Female"], label="Gender", value="Male")

            with gr.Row():
                tdee_height_input = gr.Number(label="Height Value")
                tdee_height_unit = gr.Radio(choices=["cm", "inches"], label="Height Unit", value="cm")

            with gr.Row():
                tdee_weight_input = gr.Number(label="Weight Value")
                tdee_weight_unit = gr.Radio(choices=["kg", "lbs"], label="Weight Unit", value="kg")

            activity_input = gr.Dropdown(
                choices=[
                    "Sedentary (little to no exercise)",
                    "Lightly active (1-3 days/week)",
                    "Moderately active (3-5 days/week)",
                    "Very active (6-7 days/week)",
                    "Extra active (daily, physical job)"
                ],
                label="Activity Level"
            )

            submit_tdee_button = gr.Button("Calculate TDEE")
            bmr_output = gr.Number(label="BMR")
            tdee_output = gr.Number(label="TDEE")
            tdee_lose_output = gr.Number(label="Calories to Lose Weight")
            tdee_maintain_output = gr.Number(label="Calories to Maintain")
            tdee_gain_output = gr.Number(label="Calories to Gain Weight")
            tdee_value_state = gr.State(0)

            submit_tdee_button.click(
                fn=calculate_bmr,
                inputs=[age_input, gender_input, tdee_height_input, tdee_height_unit, tdee_weight_input, tdee_weight_unit, activity_input],
                outputs=[bmr_output, tdee_output, tdee_lose_output, tdee_maintain_output, tdee_gain_output]
            ).then(lambda tdee: tdee, inputs=tdee_output, outputs=tdee_value_state)

        # TAB 3: Food Tracker
        with gr.Tab("Food Tracker"):
            gr.Markdown("### Daily Calorie Tracker")

            with gr.Row():
                food_dropdown = gr.Dropdown(choices=list(common_foods.keys()), label="Select Food (optional)")
                calorie_input = gr.Number(label="Or enter Calories manually")

            date_input_food = gr.Textbox(label="Date (YYYY-MM-DD)", value=date.today().strftime("%Y-%m-%d"))
            tdee_input_food = gr.Number(label="TDEE (from Metabolic Rate tab)", value=0, interactive=True)
            log_button = gr.Button("Log Food")

            food_log_dataframe = gr.Dataframe(headers=["Date", "Food", "Calories"], label="Food Log History")
            daily_total_output = gr.Number(label="Calories on Selected Date")
            tdee_comparison_output = gr.Text(label="Comparison to TDEE")
            weekly_plot_output = gr.Plot(label="Weekly Calorie Chart")

            log_button.click(
                fn=log_food,
                inputs=[food_dropdown, calorie_input, date_input_food, tdee_input_food],
                outputs=[food_log_dataframe, daily_total_output, tdee_comparison_output, weekly_plot_output]
            )

            demo.load(lambda tdee: tdee, inputs=tdee_value_state, outputs=tdee_input_food)

        # TAB 4: Data Trends
        with gr.Tab("Data Trends"):
            gr.Markdown("### Historical Health Data and Trends")

            bmi_table = gr.Dataframe(headers=["Date", "Height", "Unit", "Weight", "Unit", "BMI", "Category"])
            metabolic_table = gr.Dataframe(headers=["Date", "Age", "Gender", "Height", "Unit", "Weight", "Unit", "Activity", "BMR", "TDEE"])
            food_table = gr.Dataframe(headers=["Date", "Food", "Calories"])

            load_bmi_btn = gr.Button("Load BMI Data")
            load_metabolic_btn = gr.Button("Load Metabolic Data")
            load_food_btn = gr.Button("Load Food Log")

            load_bmi_btn.click(fn=get_bmi_data, inputs=None, outputs=bmi_table)
            load_metabolic_btn.click(fn=get_metabolic_data, inputs=None, outputs=metabolic_table)
            load_food_btn.click(fn=get_food_log_data, inputs=None, outputs=food_table)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6debc9e697df8dab57.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
